<a href="https://colab.research.google.com/github/RitinDev/projects-programming-data-sciences/blob/main/class3/RitinMalhotra_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 | Ritin Malhotra | rm5486
Working with NewsAPI and IBM Watson NLP Tool(s) to create a database of popular articles related to the passing of Queen Elizabeth II, and the sentiments associated with each article. 



## Workflow
1. Initialize the sentiment and emotion analysis function through IBM Watson. Since I ran into payment issues for the IBM Watson API due to not owning a U.S. based credit card, I am sharing the API key with a friend. While we will, in all probability, not run out of the daily quota for sentiment analysis, I cannot give any guarantees. If need be, the `URL` and `API_KEY` in the second code block can be changed. Apolgoies for the inconvenience.

2. Get the top headlines related to Queen Elizabeth II using NewsAPI.

3. Create a database and add the `content` and `URL` for each article to a single table. Print this table out.

4. Run sentiment analysis on each article using its `URL`. Add the sentiment analysis scores + URL to a list of dictionaries, which will enable us to easily add the values to another table. I have used Python's exception handling to ensure that a single bad article does not affect the flow of the program. I have also limited sentiment analysis to the first 100 articles to ensure that I do not consume all the IBM Watson API quota.

5. Add `sentiment_score` and `url` to a new table. Print this table out.



In [1]:
import requests

In [2]:
'''URL = 'https://api.us-south.natural-language-understanding.watson.cloud.ibm.com/instances/9e683088-0d12-4399-8118-518f3e60e8c4'

# My own API key. It may run out of quota
# You can register and get your own credentials
# The ones below have a quota of 1000 calls per day 
# and can run out quickly if multiple people use these
API_KEY = 'yx39wyiwPNGm7DoDUPCSJB4SzFkr0qurARfbGYyEdaoC'''

URL = 'https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/d56bc7f9-88a8-4b7a-aa86-0557ca745925'
API_KEY = 'xUQrt-jdAvneYY4cC7oAhLzCkS-RrDldfPfarjn9kBwl'

def analyzeText(text=None, url=None):

    endpoint = f"{URL}/v1/analyze"
    username = "apikey"
    password = API_KEY
    
    parameters = {
        'features': 'emotion,sentiment',
        'version' : '2022-04-07',
        'text': text,
        'language' : 'en',
        'url' : url # this is an alternative to sending the text
    }

    resp = requests.get(endpoint, params=parameters, auth=(username, password))
    
    return resp.json()

In [3]:
# !sudo -H pip3 install newsapi-python
!pip install newsapi-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key='a7eab21c34e545dba418c4344d59a54f')

# /v2/top-headlines
news_articles = newsapi.get_everything(q = 'queen',
                                          language='en',
                                          page_size=100)

articles = news_articles['articles']

'''for article in articles:
    article_url = article['url']
    # print(article)
    print(article['content'])
    print(article['url'])
    data = analyzeText(url=article_url)
    print(data['sentiment']['document'])
    print(data['emotion']['document'])'''

"for article in articles:\n    article_url = article['url']\n    # print(article)\n    print(article['content'])\n    print(article['url'])\n    data = analyzeText(url=article_url)\n    print(data['sentiment']['document'])\n    print(data['emotion']['document'])"

In [5]:
!sudo pip3 install -U -q PyMySQL sqlalchemy sql_magic

     |████████████████████████████████| 43 kB 1.5 MB/s 
     |████████████████████████████████| 1.6 MB 8.4 MB/s 
     |████████████████████████████████| 120 kB 40.0 MB/s 
     |████████████████████████████████| 82 kB 484 kB/s 


In [6]:
from sqlalchemy import create_engine

conn_string = "mysql+pymysql://{user}:{password}@{host}/".format(
    host="db.ipeirotis.org", user="student", password="dwdstudent2015"
)

engine = create_engine(conn_string)

In [7]:
# Query to create a database
db_name = "public"
create_db_query = (
    f"CREATE DATABASE IF NOT EXISTS {db_name} DEFAULT CHARACTER SET 'utf8'"
)

# Create a database
engine.execute(create_db_query)

In [8]:
suffix = "rm5486"
table_name = f"{suffix}_news"
# Drop any existing table
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
engine.execute(drop_table_query)
# Create a table
create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (content varchar(1000), 
                                url varchar(500), 
                                PRIMARY KEY(url)
                                )"""
engine.execute(create_table_query)

In [9]:
query_template = f"""
                    INSERT IGNORE INTO 
                    {db_name}.{table_name}(content,  url) 
                    VALUES (%s, %s)
                  """
for article in articles:
    content = article['content']
    url = article['url']

    # print("Inserting article", content, "with", url, "as URL")
    query_parameters = (content, url)
    engine.execute(query_template, query_parameters)

In [10]:
results = engine.execute(f"SELECT * FROM {db_name}.{table_name}")
rows = results.fetchall()
results.close()

In [11]:
for row in rows:
    print("Content:", row["content"])
    print("URL:", row["url"])
    print("=============================================")

Content: The death of Queen Elizabeth II, and the ascension of King Charles III as sovereign, has reanimated republican debate in Australia, as the country formally pays tribute to its longest-serving monarch… [+6808 chars]
URL: https://amp.theguardian.com/australia-news/2022/sep/09/republic-debate-flares-as-australia-mourns-its-longest-serving-monarch-queen-elizabeth-ii
Content: Queen Elizabeth II was not a huge football fan. That July 1966 afternoon, when Bobby Moore wiped himself down before shaking her gloved hand as she passed him the Jules Rimet trophy, was actually the… [+4818 chars]
URL: https://amp.theguardian.com/football/2022/sep/09/the-fiver-football-and-the-queen
Content: Once vilified as a marriage wrecker not least by Diana, Princess of Wales Queen Camilla, as she will be crowned, will take her place by the side of King Charles III at his coronation.
It had been Qu… [+2591 chars]
URL: https://amp.theguardian.com/uk-news/2022/sep/08/camilla-to-be-crowned-queen-beside-king

In [12]:
sentiment_scores = []

# Limiting sentiment analysis to a maximum of 100 articles
for row in rows[:101]:
  article_url = row["url"]
  data = analyzeText(url=article_url)
  entry = {}
  entry['url'] = article_url
  try:
    entry['score'] = data['sentiment']['document']['score']
    # print(data['sentiment']['document']['score'])
  except KeyError:
    entry['score'] = 'ERROR: Cannot determine sentiment analysis score for this article'
    # print("ERROR: Sentiment not detected")
  
  # print(entry)
  sentiment_scores.append(entry);

In [13]:
table_name = f"{suffix}_sentiment_score"
# Drop any existing table
drop_table_query = f"DROP TABLE IF EXISTS {db_name}.{table_name}"
engine.execute(drop_table_query)
# Create a table
create_table_query = f"""CREATE TABLE IF NOT EXISTS {db_name}.{table_name} 
                                (url varchar(255),
                                sentiment_score varchar(255), 
                                PRIMARY KEY(url)
                                )"""
engine.execute(create_table_query)

In [14]:
query_template = f"""
                    INSERT IGNORE INTO 
                    {db_name}.{table_name}(url, sentiment_score) 
                    VALUES (%s, %s)
                  """
for score in sentiment_scores:
    url = score['url']
    sentiment_score = score['score']

    # print("Inserting URL", url, "with", sentiment_score, "as score")
    query_parameters = (url, sentiment_score)
    engine.execute(query_template, query_parameters)

In [15]:
results = engine.execute(f"SELECT * FROM {db_name}.{table_name}")
rows = results.fetchall()
results.close()

In [16]:
for row in rows:
    print("URL:", row["url"])
    print("Sentiment Score:", row["sentiment_score"])
    print("=============================================")

URL: https://amp.theguardian.com/australia-news/2022/sep/09/republic-debate-flares-as-australia-mourns-its-longest-serving-monarch-queen-elizabeth-ii
Sentiment Score: -0.502135
URL: https://amp.theguardian.com/football/2022/sep/09/the-fiver-football-and-the-queen
Sentiment Score: -0.286043
URL: https://amp.theguardian.com/uk-news/2022/sep/08/camilla-to-be-crowned-queen-beside-king-charles-iii-at-his-coronation
Sentiment Score: 0.885948
URL: https://amp.theguardian.com/uk-news/2022/sep/08/queen-death-cash-stamps-flags
Sentiment Score: 0.451299
URL: https://amp.theguardian.com/uk-news/2022/sep/08/queen-elizabeth-ii-obituary
Sentiment Score: 0.305615
URL: https://amp.theguardian.com/uk-news/2022/sep/08/share-your-tributes-and-memories-of-queen-elizabeth-ii
Sentiment Score: 0.388746
URL: https://amp.theguardian.com/uk-news/2022/sep/08/she-has-been-so-strong-mourners-gather-at-buckingham-palace
Sentiment Score: 0.492003
URL: https://amp.theguardian.com/uk-news/2022/sep/09/constant-and-reass